## TODO: Visualization of Flow Trajectories

In [ ]:
from typing import Dict, Optional, Sequence

import numpy as np
import numpy.typing as npt
import plotly.colors as pc
import plotly.graph_objects as go

import rpad.visualize_3d.plots as v3p
from rpad.visualize_3d.primitives import pointcloud

### 1. Original static 3D flow visualization

In [ ]:
import os
from pathlib import Path

import numpy as np

import flowbot3d.datasets.flow_dataset as f3d
import rpad.partnet_mobility_utils.dataset as rpud
import rpad.pyg.dataset as rpd

In [ ]:
root = os.path.expanduser("~/datasets/partnet-mobility/raw")
split = "umpnet-train-train"

In [ ]:
# Create the dataset in online-generation mode.
dset = f3d.Flowbot3DDataset(root=root, split=split)

In [ ]:
# Generate.
data = dset[20]
flow_fig(data['pos'], data['flow'])

### 2. Dynamic 3D flow trajectory visualization

In [ ]:
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation

In [ ]:
K = 20

In [ ]:
from python_ml_project_template.datasets.flow_trajectory_dataset import FlowTrajectoryDataset
traj_dataset = FlowTrajectoryDataset(root=root, split=split, 
        randomize_joints=True,
        randomize_camera=True,
        trajectory_len=K,
        mode="both",
        n_points=1200)

In [ ]:
import torch
id = 40
traj_data = traj_dataset[id]

animation = FlowNetAnimation()
for i in range(K):
    pcd = traj_data['pos'] if i==0 else traj_data['trajectory'][:, (i-1), :3]
    mask = traj_data['mask']
    flow = traj_data['trajectory'][:, i, 3:]
    animation.add_trace(
        torch.as_tensor(pcd),
        torch.as_tensor([pcd]),
        torch.as_tensor(
            [flow]
        ),
        "red",
    )
    temp = animation.add_trace_gif(
        torch.as_tensor(pcd),
        torch.as_tensor([pcd]),
        torch.as_tensor(
            [flow]
        ),
        "red",
    )
    animation.append_gif_frame(temp)

animation.animate()

In [ ]:
import gif
gif.save(animation.gif_frames,f'./{id}.gif', duration=100)
# animation.save_gif()

In [ ]:
traj_data = traj_dataset[20]
v3p.flow_fig(traj_data['pos'], traj_data['trajectory'][:, 0, 3:])